In [1]:
# Paso 1: Importar las bibliotecas y cargar los datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score

# CLUSTERING

In [ ]:
# Cargar los datos (reemplaza 'nombre_del_archivo.csv' con el nombre de tu archivo de datos)
data = pd.read_csv('nombre_del_archivo.csv')

# Paso 2: Dividir los datos en características (X)
X = data.drop('etiqueta', axis=1)

# Paso 3: Estandarizar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Paso 4: Reducción de dimensionalidad con PCA
pca = PCA(n_components=2)  # Establecer el número de componentes principales deseadas
X_pca = pca.fit_transform(X_scaled)

# Paso 5: Elbow Plot para analizar la cantidad óptima de clusters en PCA
inertia = []
num_clusters_range = range(1, 11)
for num_clusters in num_clusters_range:
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(X_pca)
    inertia.append(kmeans.inertia_)

plt.plot(num_clusters_range, inertia, marker='o')
plt.xlabel('Número de Clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Plot para K-Means en PCA')
plt.show()

# Paso 6: Crear un pipeline para distintos algoritmos de clustering
models = [('K-Means', KMeans(n_clusters=3)),
          ('Agglomerative Clustering', AgglomerativeClustering(n_clusters=3)),
          ('DBSCAN', DBSCAN(eps=0.5, min_samples=5))]

for name, model in models:
    pipeline = Pipeline([('pca', pca), (name, model)])
    pipeline.fit(X_scaled)
    labels = pipeline.predict(X_scaled)
    
    # Paso 7: Calcular el coeficiente de silueta para evaluar el rendimiento del clustering
    silhouette_avg = silhouette_score(X_scaled, labels)
    print(f"Modelo: {name}")
    print("Coeficiente de Silueta:", silhouette_avg)

# Detección de Anomalias

In [ ]:
# Paso 1: Importar las bibliotecas y cargar los datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import roc_auc_score

# Cargar los datos (reemplaza 'nombre_del_archivo.csv' con el nombre de tu archivo de datos)
data = pd.read_csv('nombre_del_archivo.csv')

# Paso 2: Dividir los datos en características (X)
X = data.drop('etiqueta', axis=1)

# Paso 3: Estandarizar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Paso 4: Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, data['etiqueta'], test_size=0.2, random_state=42)

# Paso 5: Crear un diccionario de modelos de detección de anomalías
models = {'Isolation Forest': IsolationForest(contamination=0.1, random_state=42),
          'One-Class SVM': OneClassSVM(nu=0.1),
          'Local Outlier Factor': LocalOutlierFactor(n_neighbors=20, contamination=0.1)}

# Paso 6: Entrenar y evaluar cada modelo de detección de anomalías
results = {}
for name, model in models.items():
    model.fit(X_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Transformar las predicciones: 1 para anomalías, -1 para datos normales
    y_pred_train[y_pred_train == 1] = 0
    y_pred_train[y_pred_train == -1] = 1
    y_pred_test[y_pred_test == 1] = 0
    y_pred_test[y_pred_test == -1] = 1
    
    # Calcular el área bajo la curva ROC (AUC) para evaluar el rendimiento
    auc_train = roc_auc_score(y_train, y_pred_train)
    auc_test = roc_auc_score(y_test, y_pred_test)
    
    results[name] = (auc_train, auc_test)

# Paso 7: Imprimir los resultados de cada modelo
for name, (auc_train, auc_test) in results.items():
    print(f"Modelo: {name}")
    print("AUC en entrenamiento:", auc_train)
    print("AUC en prueba:", auc_test)
    print("-" * 50)